In [ ]:
!nvidia-smi  # gpu 확인

In [ ]:
# Google drive 연동
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

from google.colab import auth
auth.authenticate_user()

from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()

import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL

vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [ ]:
cd /

In [ ]:
# Google drive 연동 폴더 생성
!mkdir -p Gdrive
!google-drive-ocamlfuse Gdrive
!ls Gdrive

In [ ]:
# Tensorflow 1.15.0 is required (default Tensorflow version of Colab : 2.x)
!sudo pip install --upgrade pip
!sudo pip install --upgrade setuptools
!sudo pip install --upgrade virtualenv
!virtualenv --system-site-packages ~/tensorflow
!source ~/tensorflow/bin/activate
!pip install --upgrade tensorflow==1.15.0

In [ ]:
# BERT training (only once execution)
!python Gdrive/NLP/bert_chatbot/run_squad.py \
--vocab_file=Gdrive/NLP/bert_chatbot/vocab.txt \
--bert_config_file=Gdrive/NLP/bert_chatbot/bert_config.json \
--init_checkpoint=Gdrive/NLP/bert_chatbot/bert_model.ckpt \
--do_train=True \
--train_file=Gdrive/NLP/bert_chatbot/KorQuAD_v1.0_train.json \
--do_predict=False \
--predict_file=Gdrive/NLP/bert_chatbot/dev_test.json \
--train_batch_size=8 \
--learning_rate=3e-5 \
--num_train_epochs=2.0 \
--max_seq_length=128 \
--doc_stride=128 \
--output_dir=Gdrive/NLP/bert_chatbot/output_new \
--do_lower_case=False

In [ ]:
# BERT prediction(QnA)
!python Gdrive/NLP/bert_chatbot/run_squad.py \
--vocab_file=Gdrive/NLP/bert_chatbot/vocab.txt \
--bert_config_file=Gdrive/NLP/bert_chatbot/bert_config.json \
--init_checkpoint=Gdrive/NLP/bert_chatbot/bert_model.ckpt \
--do_train=False \
--train_file=Gdrive/NLP/bert_chatbot/KorQuAD_v1.0_train.json \
--do_predict=True \
--predict_file=Gdrive/NLP/bert_chatbot/dev_test.json \   # question and context should be in this file
--train_batch_size=8 \
--learning_rate=3e-5 \
--num_train_epochs=2.0 \
--max_seq_length=128 \
--doc_stride=128 \
--output_dir=Gdrive/NLP/bert_chatbot/output_new \     # answer is in predictions.json in this folder
--do_lower_case=False

In [ ]:
# Performance evaluation
!python Gdrive/NLP/bert/korquad_files/evaluate-v1.0.py Gdrive/NLP/bert/korquad_files/KorQuAD_v1.0_dev.json Gdrive/NLP/bert_chatbot/output_new/predictions.json